In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#변수명 추출
from varname import nameof
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")
            
# blnk/검색 로직

def blnk_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("blnk(null)이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
        
# 파이썬 로드후 파일 실행
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)


# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    print('예) path + "\\metadata\\" + folder + "\\01 abnormal_words_correct", "abnormal_words_correct"')
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

## 1. 전체데이터 로드

In [2]:
# 경로 및 파일명 설정
path = os.path.join(os.getcwd())
folder_list = pd.read_excel(path + "\\metadata\\철도공사-업무별-시스템 담당자 _표준화용.xlsx")['폴더관리1']
folder_list = folder_list.map(lambda x : x.split('md ')[-1])
# folder_list = folder_list[folder_list != "00_"].reset_index(drop=True)
folder_list = list(folder_list.unique())
folder = folder_list[29]
folder

'30_광역철도수입관리시스템'

In [12]:
# 모든데이터 로드
for i in folder_list:
    print(i)
    globals()["data_{}".format(i)] = pd.read_csv(path + "\\metadata\\" + i + "\\METADATA_UTF8.csv", na_filter = False)
#     globals()["data_{}".format(i)] = pd.DataFrame()

01_XROIS
02_홍보실적관리
03_한국철도TV
04_한국철도사보
05_투자사업관리
06_사옥관리
07_당직관리
08_정보공개시스템
09_인사커뮤니케이션
10_고충처리
11_복지공감방
12_코레일텔
13_철도상생플랫폼
14_물류정보시스템
15_업무용차량예약
16_법무관리시스템
17_역운영시스템
18_표준방송시스템
19_승무원홈페이지
20_예약발매시스템
21_영업관리시스템
22_수익관리시스템
23_고객센터안내시스템
24_IRIS
25_TOPS
26_열차운전안내장치
27_운행정보자동분석시스템
28_부산진CY운영시스템
29_철도물류정보시스템
30_광역철도수입관리시스템
31_광역정보센터시스템
32_RCM시스템
33_기지운용시스템
34_TIDS
35_CCTV영상정보관리
36_건설관리
37_KTX고장코드무선전송
38_사회공헌포털
39_노트북예약
40_국제철도정보시스템
41_(신)학사관리시스템
42_적성검사
43_연구개발시스템


In [14]:
data_30_광역철도수입관리시스템

,관리부서코드,관리부서명,시스템명,물리DB명,TABLE_NAME,COMMENTS,COLUMN_NAME,컬럼물리명FullName,COMMENTS.1,컬럼설명,표준도메인그룹명,도메인분류명,DATA_TYPE,DATA_LENGTH,DATA_PRECISION,DATA_SCALE
0,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DUTY_DT,,업무일자,업무일자,,,VARCHAR2,8,,
1,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_SETTM_DTIME,,정산일시,정산일시,,,VARCHAR2,14,,
2,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_EXTN_STA_NO,,외부역번호,외부역번호,,,NUMBER,22,4,0
3,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_NO,,장비번호,장비번호,,,NUMBER,22,3,0
4,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_CLASS_CD,,장비구분코드,장비구분코드,,,VARCHAR2,3,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5568,,광역철도본부,KOTRIS,US_K,VW_TT_KSR021,,STN_CD3,,,,,,VARCHAR2,4,,
5569,,광역철도본부,KOTRIS,US_K,VW_TT_KSR021,,TM_MIN,,,,,,NUMBER,22,,
5570,,광역철도본부,KOTRIS,US_K,VW_TT_KSR021,,DATA_TYPE,,,,,,CHAR,1,,
5571,,광역철도본부,KOTRIS,US_K,VW_TT_KSR021,,UPWN_DV_CD,,,,,,CHAR,1,,


In [ ]:
folder_list

In [ ]:
folder = folder_list[10]
folder

In [ ]:
# 모든테이블 union

In [ ]:
path

### 30_광역철도수입관리시스템

In [15]:
data_30_광역철도수입관리시스템.head()

,관리부서코드,관리부서명,시스템명,물리DB명,TABLE_NAME,COMMENTS,COLUMN_NAME,컬럼물리명FullName,COMMENTS.1,컬럼설명,표준도메인그룹명,도메인분류명,DATA_TYPE,DATA_LENGTH,DATA_PRECISION,DATA_SCALE
0,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DUTY_DT,,업무일자,업무일자,,,VARCHAR2,8,,
1,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_SETTM_DTIME,,정산일시,정산일시,,,VARCHAR2,14,,
2,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_EXTN_STA_NO,,외부역번호,외부역번호,,,NUMBER,22,4,0
3,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_NO,,장비번호,장비번호,,,NUMBER,22,3,0
4,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_CLASS_CD,,장비구분코드,장비구분코드,,,VARCHAR2,3,,


In [ ]:
data_31_광역정보센터시스템

In [9]:
data_30_광역철도수입관리시스템.head()

,관리부서코드,관리부서명,시스템명,물리DB명,TABLE_NAME,COMMENTS,COLUMN_NAME,컬럼물리명FullName,COMMENTS.1,컬럼설명,표준도메인그룹명,도메인분류명,DATA_TYPE,DATA_LENGTH,DATA_SCALE
0,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DUTY_DT,,업무일자,업무일자,,,VARCHAR2,8,
1,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_SETTM_DTIME,,정산일시,정산일시,,,VARCHAR2,14,
2,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_EXTN_STA_NO,,외부역번호,외부역번호,,,NUMBER,4,0
3,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_NO,,장비번호,장비번호,,,NUMBER,3,0
4,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_CLASS_CD,,장비구분코드,장비구분코드,,,VARCHAR2,3,


In [16]:
data_30_광역철도수입관리시스템['DATA_PRECISION']

0        
1        
2       4
3       3
4        
       ..
5568     
5569     
5570     
5571     
5572     
Name: DATA_PRECISION, Length: 5573, dtype: object

In [17]:
# 데이터 소수점 전처리
data_30_광역철도수입관리시스템.loc[data_30_광역철도수입관리시스템['DATA_PRECISION'] != ''] 
data_30_광역철도수입관리시스템.loc[data_30_광역철도수입관리시스템['DATA_PRECISION'] != '', 'DATA_LENGTH'] =  data_30_광역철도수입관리시스템['DATA_PRECISION']
# 컬럼명 정리
col_list = ['관리부서코드', '관리부서명', '시스템명', '물리DB명', 'TABLE_NAME', 'COMMENTS',
       'COLUMN_NAME', '컬럼물리명FullName', 'COMMENTS.1', '컬럼설명', '표준도메인그룹명',
       '도메인분류명', 'DATA_TYPE', 'DATA_LENGTH', 'DATA_SCALE']
data_30_광역철도수입관리시스템 = data_30_광역철도수입관리시스템[col_list]
# data_30_광역철도수입관리시스템.columns = smpl_col
data_30_광역철도수입관리시스템.head()

,관리부서코드,관리부서명,시스템명,물리DB명,TABLE_NAME,COMMENTS,COLUMN_NAME,컬럼물리명FullName,COMMENTS.1,컬럼설명,표준도메인그룹명,도메인분류명,DATA_TYPE,DATA_LENGTH,DATA_SCALE
0,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DUTY_DT,,업무일자,업무일자,,,VARCHAR2,8,
1,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_SETTM_DTIME,,정산일시,정산일시,,,VARCHAR2,14,
2,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_EXTN_STA_NO,,외부역번호,외부역번호,,,NUMBER,4,0
3,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_NO,,장비번호,장비번호,,,NUMBER,3,0
4,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_CLASS_CD,,장비구분코드,장비구분코드,,,VARCHAR2,3,


In [ ]:
smpl_col = [] # 정의해야!!! 샘플데이터 가지고!!

In [ ]:
for i in zip(range(len(folder_list)), folder_list):
    print(i)
#     print(j)

In [ ]:
for i in folder_list:
    lead
    print(i)

In [ ]:
# 2.중복제거 후 형태이상단어 검색

# 형태이상 단어 탐색

In [ ]:
# 경로 및 파일명 설정
path = os.path.join(os.getcwd())
folder_list = pd.read_excel(path + "\\metadata\\폴더명.xlsx")['폴더명']
folder = folder_list[10]
folder

In [ ]:
# 파일 불러오기
data_name = "METADATA_UTF8.csv"
sheet_name = ""
input_meta = pd.read_csv(path + "\\metadata\\" + folder + "\\" + data_name, na_filter=False)
print(input_meta.shape)
input_meta.head(5)

### 시스템마다 전처리 로직

In [ ]:
# 데이터 소수점 전처리
input_meta.loc[input_meta['DATA_PRECISION'] != ''] 
input_meta.loc[input_meta['DATA_PRECISION'] != '', 'DATA_LENGTH'] =  input_meta['DATA_PRECISION']
# 컬럼명 정리
col_list = ['관리부서코드', '관리부서명', '시스템명', '물리DB명', 'TABLE_NAME', 'COMMENTS',
       'COLUMN_NAME', '컬럼물리명FullName', 'COMMENTS.1', '컬럼설명', '표준도메인그룹명',
       '도메인분류명', 'DATA_TYPE', 'DATA_LENGTH', 'DATA_SCALE']
input_meta = input_meta[col_list]
input_meta.columns = smpl_col
input_meta.head()

In [ ]:
input_meta

In [ ]:
'aaa'.islower()

In [ ]:
'Aaa'.islower() == False

In [ ]:
'Aaa'.islower()

In [ ]:
'aa-_a'.islower()

In [ ]:
'Aaa'.isupper()

In [ ]:
'가나다'.isupper()

In [ ]:
'가a나'.isalpha()

In [ ]:
'Aaa'.isalpha()

In [ ]:
'가나다aAa'.isalpha()

In [ ]:
'가나다aAa'.islower()

In [ ]:
'가나다aAa'.isupper()

In [ ]:
input_meta.loc[0, 'stwd_kor'].islower()

In [ ]:
''.join(re.findall('[A-Z0-9가-힣_+]', input_meta.loc[0, '컬럼한글명*']))

In [ ]:
input_meta.loc[0, '컬럼한글명*']

In [ ]:
### 변경 now- 이거야!<단순화>
def wd_emoti_rmv(data, col, var, kor_chk_list):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        globals()["{}".format(kor_chk_list)] = []
        for i in tqdm(range(len(data))):
            
            # 대문자 변경
            data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()
            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
#             print("-----------------------------영문대문자, 숫자, 한글만 허용----------------------------|")
            print('index: ' + str(i))
            print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)
            
        # 변경 리스트 저장
        globals()["{}".format(kor_chk_list)] = data[data['컬럼한글명*'] != data['stwd_kor']].index
        cnt = len(data[data['컬럼한글명*'] != data['stwd_kor']].index)
        
        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print(kor_chk_list)
        print("자동 처리: "+ str(cnt))
        print('[자동 처리 완료하였습니다]')

In [ ]:
wd_emoti_rmv(input_meta, "컬럼한글명*", 'kor', 'chk_kor_list')

### 한글명 미입력 처리 제외 안하고 그냥 가기

In [ ]:
# input_meta = input_meta[input_meta['테이블한글명*'] != '']
print(input_meta.shape)
input_meta.head()

### 중복제거후 수정

In [ ]:
len(input_meta.groupby(['stwd_kor']).head(1)['stwd_kor'])

In [ ]:
input_meta['stwd_spce'] = input_meta['stwd_kor']
input_meta.head()

In [ ]:
input_meta.groupby(['stwd_kor']).head(1)[['stwd_kor', 'stwd_spce']]

In [ ]:
# read & load

# 파이썬 로드후 파일 실행
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)


# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    print('예) path + "\\metadata\\" + folder + "\\01 abnormal_words_correct", "abnormal_words_correct"')
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [ ]:
input_meta['테이블한글명*'].unique()

In [ ]:
input_meta['stwd_kor'].unique()

In [ ]:
save_grip_file(input_meta.loc[chk_kor_list], )

In [ ]:
print(len(input_meta.loc[chk_kor_list]))
input_meta.loc[chk_kor_list]

### 한글명 null 없애기(수기로)

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

In [ ]:
# 표준용어 구분자 변환 점검
print(len('가 나 다'.split(" ")))
'가 나 다'.split(" ")

In [ ]:
'가 나 다'.count(" ")

# 한글명은 notNull이어야 함

작업내용:
1. 중복제거(unique): 작업 효율성 제고
2. 개수 매칭 -> (모든 값 채우기)
3. 순서 매칭: ID 부여, 한글/영문
4. 재조립( 한글명 기준)

In [ ]:

테이블 한글명 + 컬럼 한글명 한줄 생성
도메인 한글명 - > 도메인명 생성

# 용어분리

In [ ]:
# 파일 불러오기
data_name = "input_test_KORAIL"
sheet_name = "테이블컬럼 입력항목 통계조사"
input_path = os.path.join(os.getcwd(), "input\\")
output_path = os.path.join(os.getcwd(), "output\\")
input_meta = pd.read_excel(input_path + data_name +".xlsx", sheet_name=sheet_name, header=0, na_filter=False)

In [ ]:
# 테이블 따로(명명규칙)
input_meta['분리된테이블한글명'] = input_meta['테이블한글명*']

# 컬럼 따로(명명규칙)
input_meta['분리된컬럼한글명'] = input_meta['컬럼한글명*']

# 컬럼명부터

In [ ]:
input_meta_sepa_col = pd.DataFrame(input_meta['분리된컬럼한글명'])
input_meta_sepa_col

In [ ]:
input_meta_sepa_col_unq = input_meta_sepa_col['분리된컬럼한글명'].unique()
input_meta_sepa_col_unq

In [ ]:
zip(input_meta_sepa_col_unq, input_meta_sepa_col_unq)

In [ ]:
a = pd.DataFrame(zip(input_meta_sepa_col_unq,input_meta_sepa_col_unq), columns = ['원본', '구분자입력'])
a


In [ ]:
input_meta_sepa_col

In [ ]:
input_meta_sepa_col.merge(on=a, how='left', left_on='분리된컬럼한글명', right_on='원본')

In [ ]:
pd.DataFrame([input_meta_sepa_col_unq, input_meta_sepa_col_unq])

In [ ]:
pd.DataFrame(zip(input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq), columns = ['원본', '구분자입력'])

In [ ]:
pd.DataFrame([input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq])

In [ ]:
input_meta_sepa_tbl

In [ ]:
input_meta_sepa_tbl.iloc[:,0]

In [ ]:
pd.DataFrame(columns = input_meta_sepa_tbl.columns)

In [ ]:
input_meta_sepa_tbl = input_meta_sepa_tbl.fillna('')

In [ ]:
emoti_rmv(input_meta_sepa_tbl, '분리된테이블한글명', 'kor')

In [ ]:
input_meta_sepa_tbl['stwd_kor'] = input_meta_sepa_tbl['분리된테이블한글명']
input_meta_sepa_tbl['stwd_kor']

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')
        

        print(data)
        print(1)

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
# # 파일 불러오기
# sheet_name = "테이블컬럼정의서"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# input_meta = pd.read_excel(input_path+"표준화 INPUT 정의서.xlsx", sheet_name=sheet_name, header=0, na_filter=False)

# # null string ""로 변환
# for i in range(len(input_meta.columns)):
#     input_meta[input_meta.columns[i]] = input_meta[input_meta.columns[i]].fillna("")
# input_meta['기관ID'] = input_meta['물리DB명*'] + "_" + input_meta['관리부서명*']
# print(input_meta.shape)
# input_meta.head(5)

In [ ]:
input_meta

### 값 검색 로직

In [ ]:
# 널 체크
blnk_chk(input_meta)

In [ ]:
input_meta_tbl

In [ ]:
len(input_meta_tbl.columns)

In [ ]:
# 테이블 유니크 선벌(그룹바이)
input_meta_tbl = input_meta.groupby(['테이블물리명*', '테이블한글명*']).head(1)
# 불필요 값 없애기

for i in range(6,14):
    input_meta_tbl.iloc[:,i] = ''
input_meta_tbl

In [ ]:
# 불필요 컬럼 빼기
excpt_col = ['컬럼물리명*', '컬럼영문약어명*', '컬럼한글명*', '분리된컬럼한글명*']
excpt_tbl = ['테이블물리명*', '테이블영문약어명*', '테이블한글명*', '분리된테이블한글명*']

# 컬럼명 추림
brief_col(input_meta_tbl, excpt_col, 'pick_tbl')
brief_col(input_meta, excpt_tbl, 'pick_col')

# 용어명 컬럼 추가
pick_tbl['용어명'] = pick_tbl['테이블한글명*']
pick_col['용어명'] = pick_col['컬럼한글명*']
pick_tbl

In [ ]:
# 컬럼명 변경
modify_col = ['물리DB명*', '관리부서명*', '테이블물리명*', '영문약어명', '테이블한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_tbl, pick_tbl.columns, modify_col, 'pick_tbl')
modify_col = ['물리DB명*', '관리부서명*', '컬럼물리명*', '영문약어명', '컬럼한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_col, pick_col.columns, modify_col, 'pick_col')

# TBL/COL 기록
pick_col["TBL/COL"] = "COL"
pick_tbl["TBL/COL"] = "TBL"

In [ ]:
# null 제거 및 컬럼(순서) 선택
pick_total = pd.concat([pick_col, pick_tbl]).fillna("")
pick_total_col = ['물리DB명*', '관리부서명*',  '테이블물리명*', '테이블한글명*', '컬럼물리명*', '컬럼한글명*', 
                  '용어명', '용어명 분리', '영문약어명', 
                  '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', 'TBL/COL']
cor_col(pick_total, pick_total_col, pick_total_col, 'pick_total')
# 인덱스 초기화
pick_total = pick_total.reset_index(drop=True)
print(pick_total.shape)
pick_total.head()

#### . 파일 불러오기

In [ ]:
# # 파일 불러오기
# # test_emoti_dup_korDup
# sheet_name = "input_test"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# # input_test = pd.read_excel(input_path+"test_emoti_dup_korDup.xlsx", sheet_name=sheet_name, header=0)
# input_test = pd.read_excel(input_path+"test_final.xlsx", sheet_name=sheet_name, header=0)
# # test_final.xlsx
# # null 제거
# for i in range(len(input_test.columns)):
#     input_test[input_test.columns[i]] = input_test[input_test.columns[i]].fillna("")
# print(input_test.shape)
# input_test.head(5)

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

# 2. 용어 분리

### 용어명 변경

In [ ]:
input_meta_spce = pick_total.copy()
input_meta_spce['영문약어명'] = ''
input_meta_spce['용어명 분리'] = ''
input_meta_spce.head()

In [ ]:
rmv_spce(input_meta_spce, 'col', '용어명')
rmv_spce(input_meta_spce, 'col', '영문약어명')
rmv_spce(input_meta_spce, 'col', '용어명 분리')

In [ ]:
컬럼한글명 유니크!! - 용어명이 KEY값임
out 후 적용
수정 후 적용 - 한글명기준(인덱스 무시)
중복 검사
1. 컬럼한글명 유니크 수 == 컬럼한글명&&분리된컬럼한글명 유니크 수
# ->단어 변환 후 검사 노!!!!!!!!! 여기서는 1차 분리임!!!!!!!!!

검사?

### 컬럼한글명 유니크 생성

In [ ]:
input_meta_dict = input_meta_spce.groupby('용어명').head(1)[['용어명','용어명 분리', '영문약어명', 'TBL/COL']].reset_index(drop=True)
print(input_meta_dict.shape)
input_meta_dict.head()

In [ ]:
# 저장
input_meta_dict.to_excel(output_path + 'input_meta_space.xlsx')

In [ ]:
input_meta_dict = pd.read_excel(output_path + 'input_meta_space.xlsx', header=0, index_col=0, na_filter=False)
print(input_meta_dict.shape)
input_meta_dict.head()

In [ ]:
cmpar(input_meta_dict, '용어명', '용어명 분리')
cmpar(input_meta_dict, '용어명', '영문약어명')

# 분리 시작

In [ ]:
# 인터페이스 No
input_test = pick_total.copy()
input_test['No'] = [*range(len(input_test))]
input_test.columns

In [ ]:
# 인덱스 DF 초기화
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

In [ ]:
# 용어명 분리

# 용어명 분리
input_split_kor = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist()).fillna("")

# 영문명 없을 경우 대비하여 cnt 집계
kor_firts_cnt = len(input_split_kor.columns)

# 컬럼명 입력
col_list_kor = []
for i in range(len(input_split_kor.columns)):
    col_list_kor.append("stwd_kor{}".format(i))
input_split_kor.columns = col_list_kor

# 한글 개수 세기
dupl_list_kor = []
dupl_cnt_kor = []
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = []
    for j in range(len(input_split_kor["stwd_kor{}".format(i)])):
        dupl_list_kor = input_split_kor["stwd_kor{}".format(i)].to_list().count(input_split_kor["stwd_kor{}".format(i)][j])
        globals()["dupl_list_kor{}".format(i)].append(dupl_list_kor)
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_kor{}".format(i)])], axis = 0)
    globals()["dupl_list_kor{}".format(i)].columns = ["dupl_list_kor{}".format(i)]
    
# 컬럼명,index, 단어위치 추가
for i in range(len(input_split_kor.columns)):
    input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
    globals()["stwd_kor{}".format(i)] = pd.concat(
            [
            df_index.fillna(""), 
            pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
            globals()["dupl_list_kor{}".format(i)].fillna("")
            ], 
            axis=1)
print(col_list_kor)
print(kor_firts_cnt)
# stwd_kor9
# dupl_list_kor0

In [ ]:
# 영문명 분리

# 영문명이 하나라도 입력돼 있을 경우
if len(input_test['영문약어명']) > 0:
    print("영문명 컬럼 수 1개 이상:"+str((input_test['영문약어명']=="").sum()))

    # 영문명 분리
    input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist()).fillna("")
    
    # 영문명 컬럼 수
    eng_firts_cnt = len(input_split_eng.columns)
    
    # 컬럼명 입력
    col_list_eng = []
    for i in range(len(input_split_eng.columns)):
        col_list_eng.append("stwd_eng{}".format(i))
    input_split_eng.columns = col_list_eng
    
    # 영문 개수 세기
    dupl_list_eng = []
    dupl_cnt_eng = []
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = []
        for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
            dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
            globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
        globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]
        
    # 컬럼명,index, 단어위치 추가
    for i in range(eng_firts_cnt):
        input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)] = pd.concat(
                [
                df_index.fillna(""), 
                pd.DataFrame(input_split_eng["stwd_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                globals()["dupl_list_eng{}".format(i)].fillna("")
                ], 
                axis=1)
        
# 영문명이 없을 경우
elif len(input_test['영문약어명']) == 0:
    print("영문명 컬럼 수 0개:"+str((input_test['영문약어명']=="").sum()))
    
    col_list_eng = []
    for i in range(kor_firts_cnt):
        globals()["stwd_eng{}".format(i)] = pd.DataFrame(columns = ["stwd_eng{}".format(i)])
        globals()["num_eng{}".format(i)] = pd.DataFrame(columns = ["num_eng{}".format(i)])
        globals()["dupl_list_eng{}".format(i)] = pd.DataFrame(columns = ["dupl_list_eng{}".format(i)])
        globals()["stwd_eng{}".format(i)] = pd.concat(
            [
                df_index.fillna(""), 
                globals()["stwd_eng{}".format(i)].fillna(""),
                globals()["num_eng{}".format(i)].fillna(""),
                globals()["dupl_list_eng{}".format(i)].fillna(""),
            ]
            , axis = 1
        )
        globals()["stwd_eng{}".format(i)]["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)]["dupl_list_eng{}".format(i)] = len(globals()["stwd_eng{}".format(i)])
        col_list_eng.append("stwd_eng{}".format(i))
        
else:
    print("한글컬럼, 영문커럼 예외 사항")

# 컬럼수 일치 점검
col_cnt_kor = len(col_list_kor)
col_cnt_eng = len(col_list_eng)
if len(col_list_kor) == len(col_list_eng):
    print("한글컬럼수와 영문컬럼수 일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
else:
    print("한글컬럼수와 영문컬럼수 불일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))                                                       
print(col_list_eng)
stwd_eng0

In [ ]:
# 한글/영문 컬럼 수 일치시키기

# 한글컬럼과 영문컬럼 수 일치할 경우
if col_cnt_kor == col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = None
    col_cnt_diff = None
    print('한글컬럼과 영문컬럼 수 일치')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
    
# 한글컬럼이 더 많을 경우
elif col_cnt_kor > col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = list(set(range(col_cnt_kor)) - set(range(col_cnt_eng)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        input_split_eng['dupl_list_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["dupl_list_eng{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('한글컬럼이 더 많은 싱태')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  

# 영문컬럼이 더 많을 경우
elif col_cnt_kor < col_cnt_eng:
    col_cnt = col_cnt_eng
    col_cnt_diff_list = list(set(range(col_cnt_eng)) - set(range(col_cnt_kor)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        input_split_eng['dupl_list_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)].fillna("")).
                pd.DataFrame(input_split_eng["dupl_list_kor{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('영문컬럼이 더 많은 싱태')       
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
stwd_eng3

In [ ]:
# n x m 정사각형 DafaFrame 점검
for i in range(col_cnt_kor):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_kor = round(np.mean(avg), 2)
for i in range(col_cnt_eng):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_eng = round(np.mean(avg), 2)
stwd_total_cnt = int(avg_avg_eng)
if avg_avg_kor == avg_avg_eng:
    print("한글 영문 row수 일치: 평균=" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))
else:
    yes_or_no("한글명과 영문명의 row count가 일치하지 않습니다. 계속 진행하시겠습니까?")
    print("한글 영문 row수 불일치: 평균" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))

In [ ]:
# 중복검사를 위한 left join
for i in range(col_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )
print(col_cnt)
# stwd_total9

In [ ]:
# 일렬 세우기

# unique_stwd 초기화 
stwd_total = pd.DataFrame()

## 컬럼명 바꾸고  concat
for i in range(col_cnt):
    globals()["stwd_total{}".format(i)].columns = [
        'index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'stwd_eng','num_eng', 'dupl_list_eng'
    ]
    stwd_total = pd.concat([stwd_total, globals()["stwd_total{}".format(i)]], axis = 0, ignore_index = True)
print(stwd_total.shape)
stwd_total

In [ ]:
nll_chk(stwd_total)

# 3. 이상값 수정

### 3.1. 한글명에 한글과 영문대문자, 숫자 이외 표시 및 삭제 로직

In [ ]:
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):

        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
# 데이터 검토
ans = yes_or_no("데이터 검토를 실행 하시겠습니까?")
if ans == True:
    emoti_chk('kor')
    emoti_chk('eng')
elif ans == False:
    print("계속 진행하겠습니다")

### 3.2. 특수문자 제거 참고

In [ ]:
# 변경 체크 로직

s1 = '고구마acAB1!$%@! 32'
s2 = '깎ㅉ니'
s3 = "RR깍찌니12"
s4 = "123깍찌니aS"
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s1))
print(str(len(re.findall('[A-Z0-9가-힣+]', s1))) + " vs " + str(len(list(s1))))
print(len(re.findall('[A-Z0-9가-힣+]', s1)) == len(list(s1)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s2))
print(str(len(re.findall('[A-Z0-9가-힣+]', s2))) + " vs " + str(len(list(s2))))
print(len(re.findall('[A-Z0-9가-힣+]', s2)) == len(list(s2)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s3))
print(str(len(re.findall('[A-Z0-9가-힣+]', s3))) + " vs " + str(len(list(s3))))
print(len(re.findall('[A-Z0-9가-힣+]', s3)) == len(list(s3)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s4))
print(str(len(re.findall('[A-Z0-9가-힣+]', s4))) + " vs " + str(len(list(s4))))
print(len(re.findall('[A-Z0-9가-힣+]', s4)) == len(list(s4)))
print("------------------------------------------------------")

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
dict_insp = pd.read_csv(path_dict + '/dict_total/' + 'dict_insp.csv', index_col=0, na_filter = False)
print(dict_insp.shape)
dict_insp

In [ ]:
dict_list = dict_insp.iloc[:,0]
dict_list

In [ ]:
dict_insp[dict_insp['어휘'] == '등급']

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
dict_insp_oneMore = pd.read_csv(path_dict + '/dict_total/' + 'dict_expn_oneMore.csv', index_col=0, na_filter=False)
print(dict_insp_oneMore.shape)
dict_insp_oneMore

In [ ]:
dict_insp[dict_insp['어휘'] =='라']

# 국어사전과 ISIN 비교

# 사전에 없는 단어 목록 분리

In [ ]:
# 사전에 있는 단어 목록
dict_is = stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]
dict_is

In [ ]:
# 사전에 없는 단어 목록
dict_nan = stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘']) == False]
dict_nan

In [ ]:
dict_nan_unq = pd.DataFrame(dict_nan['stwd_kor'].unique(), columns = ['bf_stwd_kor'])
dict_nan_unq['af_stwd_kor'] = ''
dict_nan_unq

In [ ]:
# dict_isin 초기화
def dict_isin_init(data):
    data['dict_chk'] = 'N'
    data['notin_wrn'] = ''
    print('dict_chk 컬럼이 초기화 되었습니다.')

dict_isin_init(dict_nan_unq)
dict_nan_unq

# 저장

In [ ]:
# 저장
dict_nan_unq.to_excel(output_path + "dict_nan_unq.xlsx")

# 파이썬 로드, 파일열기 및 수기수정

In [ ]:
# 파이썬 로드후 파일로드
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)

In [ ]:
# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [ ]:
# 로드 및 파일실행
read_grip_file(dict_nan_unq, output_path, "dict_nan_unq")
dict_nan_unq

1. dict_chk = 'Y'
2. af_stwd_kor = isin(사전List)
3. ''

In [ ]:
# 가입뿅 테스트
dict_nan_unq.loc[dict_nan_unq['af_stwd_kor'] == '가입', 'af_stwd_kor'] = '가입뿅'
dict_nan_unq

# (수기수정 완료했다면) 로드 후=점검~~~~~~~~

In [ ]:
def chk_dict_isin(data, c_list):
    chk_Y = data.loc[data['dict_chk']=='Y', 'dict_chk']
    chk_N = data.loc[data['dict_chk']=='N', 'dict_chk']
    # 모두 dict_chk(Y) 일 경우통과
    if chk_Y.count() == data['dict_chk'].count():
        print("모두 수정되었습니다 "+'dict_chk'+" count(Y): " + str(chk_Y.count()))
         # 모두 notin_wrn(Y) 일 경우통과
        chk_notin_wrn = data.loc[~data['af_stwd_kor'].isin(c_list), 'notin_wrn'] 
        data['notin_wrn'] = ''
        if len(chk_notin_wrn) > 0:
            data.loc[~data['af_stwd_kor'].isin(c_list), 'notin_wrn'] = 'Y'
            print("[경고] 사전에 등록되어 있지 않은 단어가 존재합니다")
            print(data.loc[data['notin_wrn'] == 'Y', 'af_stwd_kor'])
        else:
            print("모든 단어가 사전에 등록되어 있습니다.")
    # dict_chk(N) 이 한개라도 있을 경우 반려
    elif chk_N.count() > 0:
        print("[경고] 아직 수정하지 않은 데이터가 있습니다 "+'dict_chk'+" count(N): " + str(chk_N.count()))

In [ ]:
# 가입뿅 테스트
dict_nan_unq.loc[dict_nan_unq['af_stwd_kor'] == '가입뿅', 'af_stwd_kor'] = '가입'
dict_nan_unq

In [ ]:
chk_dict_isin(dict_nan_unq, dict_list)
dict_nan_unq

In [ ]:
# 로드, 저장 및 파일 실행
save_grip_file(dict_nan_unq, output_path, "dict_nan_unq")

# 사전에 대입~ bf_stwd 를 af_stwd로

In [ ]:
# dict_nan에서 사전단어 수정 후 stwd 적용
before = dict_nan_unq['bf_stwd_kor']
after = dict_nan_unq['af_stwd_kor']

for i, j in zip(before, after):
    dict_nan.loc[dict_nan['stwd_kor'] == i, 'stwd_kor'] = j
dict_nan

# ISIN 점검

In [ ]:
# 사전 등록 점검(리스트 필요)
def dict_word_isin(data, c_list, col_name):
    # ''(공란) 제거
    data = data[data[col_name] != '']
    if (data[col_name].isin(c_list)==False).sum() == 0:
        print("모든 단어가 사전에 등록되어 있습니다.")

In [ ]:
dict_word_isin(dict_nan, dict_list, 'stwd_kor')

In [ ]:
dict_nan[(dict_nan['stwd_kor'].isin(dict_list)==False)]

In [ ]:
dict_nan['stwd_kor'].unique()

In [ ]:
dict_nan[dict_nan['stwd_kor'].isin(dict_list)]

In [ ]:
dict_list[dict_list == '가입']

In [ ]:
from varname import nameof

In [ ]:
nameof(stwd_total)

In [ ]:
def chk_mrg_idx(data1, data2, cp_data):
    rw_data = pd.concat([data1, data2]).sort_index()
    cnt = (cp_data.index == rw_data.index).sum()
    cnt_incs = (cp_data.index != rw_data.index).sum()
    # merge 전 index 목록과 다른것 비교
    if cnt_incs == 0:
        print("merge 전/후의 index가 같습니다. count:"+str(cnt))
        globals()["{}".format(nameof(stwd_total))] = rw_data
        print("수정 내용을 적용하였습니다 (변수명:"+ nameof(stwd_total) + ")")
    elif cnt_incs > 0:
        print("merge 전/후의 index가 다릅니다.count:"+str(cnt))


In [ ]:
chk_mrg_idx(dict_is, dict_nan, stwd_total)

In [ ]:
stwd_total

# 재로드~

In [ ]:
dict_nan_unq.to_excel(output_path + "dict_nan_unq.xlsx")

In [ ]:
dict_nan_unq = dict_nan['stwd_kor'].unique()
dict_nan_unq

In [ ]:
# 사전에 있는 단어 검색

stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]

In [ ]:
a = pd.concat([
stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘']) == False],stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]
]).sort_index()
a

In [ ]:
(stwd_total['stwd_kor'] == a['stwd_kor']).sum()

In [ ]:
# 사전에 없는 단어 검색
def isin_dict(data, cpr_data, data_col, cpr_data)
    data[data[data_col].isin(cpr_data[cpr_data]) == False]

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
test = pd.read_csv(path_dict + '/dict_total/' + 'dict_expn_dup.csv', index_col=0)
test = test[test['어휘'].notna()]
print(test.shape)
test

In [ ]:
dict_insp_oneMore.loc[dict_insp_oneMore['어휘'] == '개설']

In [ ]:
test.loc[test['어휘'] == '개설']

In [ ]:
stwd_total[stwd_total['stwd_kor'].isin(test['어휘'])]

In [ ]:
stwd_total[stwd_total['stwd_kor'].isin(dict_insp_list)==False]

In [ ]:
for i in tqdm(dict_insp_list):
    print(i)

In [ ]:
stwd_total

In [ ]:
nll_to_blnk(stwd_total, 'all')

# 4. 공통 표준 우선 적용

In [ ]:
nll_to_blnk(stwd_total, 'all')

In [ ]:
def pub_stnd(data):
    if yes_or_no("영문 약어명을 공통 표준으로 통일 하시겠습니까?"):

        # 표준 파일 불러오기
        sheet_name = "공통표준단어"
        input_path = os.path.join(os.getcwd(), "input\\")
        public_stwd = pd.read_excel(input_path + "공공데이터 공통표준용어(2021.12월).xlsx"
                                    , sheet_name=sheet_name
                                    , header=0
                                    , index_col="NO")
        public_stwd = public_stwd[['공통표준단어명', '공통표준단어영문약어명']]

        # 표준 영문명 자동 기입(널 제외)

        # 표준 단어명 기준 left join (영문명이 null이면 비표준, notnull이면 표준)
        std_chk = data.merge(public_stwd, left_on = 'stwd_kor', right_on = '공통표준단어명', how = 'left')
        for i in range(std_chk.shape[0]):

            # joined DF의 영문명이 있을 경우 표준으로 대신 기입 
            notna_chk = std_chk['stwd_eng'].notna()[i]
            if notna_chk:
                std_chk.loc[i, 'stwd_eng'] = std_chk.loc[i, '공통표준단어영문약어명']

        std_chk_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
        globals()["std_chk"] = std_chk[std_chk_list]

        print('한글명을 기준으로 공통(행안부) 표준을 우선 적용합니다.')
        print(std_chk.shape)
        print('OUT: std_chk')
#         print(std_chk.head())
    else:
        print('공통(행안부) 표준을 우선 적용하지 않고 표준화를 진행합니다.')    

In [ ]:
pub_stnd(stwd_total)
std_chk.tail()

In [ ]:
pub_stnd(row_blnk_stwd_kor)

In [ ]:
pub_stnd(stwd_total)
std_chk.tail()

In [ ]:
std_chk.to_excel(input_path + 'aa.xlsx')

In [ ]:
# if yes_or_no("영문 약어명을 공통 표준으로 통일 하시겠습니까?"):

#     # 표준 파일 불러오기
#     sheet_name = "공통표준단어"
#     input_path = os.path.join(os.getcwd(), "input\\")
#     public_stwd = pd.read_excel(input_path + "공공데이터 공통표준용어(2021.12월).xlsx", sheet_name=sheet_name, header=0)
#     public_stwd = public_stwd[['NO', '공통표준단어명', '공통표준단어영문약어명']]

#     # 표준 영문명 자동 기입(널 제외)

#     # 표준 단어명 기준 left join (영문명이 null이면 비표준, notnull이면 표준)
#     std_chk = stwd_total.merge(public_stwd, left_on = 'stwd_kor', right_on = '공통표준단어명', how = 'left')
#     for i in range(std_chk.shape[0]):
        
#         # joined DF의 영문명이 있을 경우 표준으로 대신 기입 
#         notna_chk = std_chk['stwd_eng'].notna()[i]
#         if notna_chk:
#             std_chk.loc[i, 'stwd_eng'] = std_chk.loc[i, '공통표준단어영문약어명']

#     std_chk_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
#     std_chk = std_chk[std_chk_list]
    
#     print('한글명을 기준으로 공통(행안부) 표준을 우선 적용합니다.')
# else:
#     print('공통(행안부) 표준을 우선 적용하지 않고 표준화를 진행합니다.')
    
# print(std_chk.shape)
# std_chk.tail()
# std_chk.head()

# 5. 한/영  중복제거 및 통합

In [ ]:
# 한글명 중복 검사: 수기 수정 후 최대 unique 값 카운트
def find_unique_eng(df):
    
    kor_input_cnt = sum(df_step_1['stwd_kor'].notna())
    kor_unique_cnt = len(set(df_step_1['stwd_kor']))
    eng_input_cnt = sum(df_step_1['stwd_eng'].notna())
    eng_unque_cnt = len(set(df_step_1['stwd_eng']))
    
    print("==중복값 비교==")
    # (한)미입력값 개수
    print('(한)     not null   개수: ' + str(kor_input_cnt))
    # (한)unique 값 개수
    print("(한)''포함 unique값 개수: " + str(kor_unique_cnt)) 
    # (영)미입력값 개수
    print('(영)     not null   개수: ' + str(eng_input_cnt))
    # (영)unique 값 개수
    print("(영)''포함 unique값 개수: " + str(eng_unque_cnt)) 
    
    if (kor_input_cnt == eng_input_cnt) and (kor_unique_cnt == eng_unque_cnt):
        print('"한글명과 영문명의 unique 개수가 같습니다."')
    else:
        print('"한글명과 영문명의 unique 개수가 다릅니다."')

In [ ]:
def chk_unique(data):
    global df_step_1   
    df_step_1 = data
    # 중복제거 후 다시 합쳐서 검사(1안)

    # 한글 중복제거
    col_stwd_kor = ['stwd_kor', 'num_kor']
    unique_kor = df_step_1['stwd_kor'].drop_duplicates()
    stwd_kor_unique = df_step_1.loc[unique_kor.index][col_stwd_kor]
    print("한글명중복제거:"+ str(stwd_kor_unique.shape))
    # stwd_kor_unique

    # 영문 중복제거
    col_stwd_eng = ['stwd_eng', 'num_eng']
    unique_eng = df_step_1['stwd_eng'].drop_duplicates()
    stwd_eng_unique = df_step_1.loc[unique_eng.index][col_stwd_eng]
    print("영문명중복제거:"+ str(stwd_eng_unique.shape))
    # stwd_eng_unique.tail(100)

    # 한글+영문 일렬 세우기
    df_step_1 = pd.DataFrame(stwd_total['index'])
    df_step_1 = df_step_1.merge(
        stwd_kor_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    ).merge(
        stwd_eng_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    )
    print(df_step_1.shape)
    df_step_1

    df_step_1.to_excel(input_path + "df_step_1" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S')) + ".xlsx")
    print(df_step_1.shape)
    df_step_1
    print("OUT: df_step_1")
    
chk_unique(std_chk)
find_unique_eng(df_step_1)
df_step_1

In [ ]:
nll_to_blnk(df_step_1, 'all')
df_step_1

In [ ]:
blnk_row_rmv(df_step_1, 'col', 'stwd_kor')
blnk_rmv_stwd_kor.tail()

In [ ]:
blnk_rmv_stwd_kor.to_excel(output_path + "blnk_rmv_stwd_kor.xlsx")

In [ ]:
df_step_1

In [ ]:
data

In [ ]:
data.iloc[i]

In [ ]:
data = df_step_1
data_cnt = len(data)
notna_list = []
for i in range(data_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if data.iloc[i].notna().sum() > 1:
        notna_list.append(i)
        notna_list

In [ ]:
row_nll_rmv(df_step_1, 'df_step_1')

In [ ]:
df_step_1

# * 한글/영문 단어수 매칭

## 6. 자동수정, 파일 업로드 점검

### 6.1.수기 수정값 표시(! y/n)

In [ ]:
### 값이 한개라도 있을 경우만 카운트 ###

df_step_1_cnt = len(df_step_1)
notna_list = []
for i in range(df_step_1_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_step_1.iloc[i].notna().sum() > 1:
        notna_list.append(i)

df_step_1 = df_step_1.loc[notna_list]
print(df_step_1.shape)

# 원래 표준과 병합(수기 수정을 위해)
df_step_1 = df_step_1.merge(
    input_test,
    left_on = 'index',
    right_on = 'No',
    how = 'left'
)

# 영문 개수 세기
dup_list_eng = []
dup_cnt_eng = []
for i in range(df_step_1.shape[0]):
    dup_list_eng = df_step_1['stwd_eng'].to_list().count(df_step_1.loc[i, 'stwd_eng'])
    dup_cnt_eng.append(dup_list_eng)
df_step_1 = pd.concat([df_step_1, pd.DataFrame(dup_cnt_eng, columns = {"eng_cnt"})], axis = 1)

# 컬럼 추리기
df_step_1_list = ['index', 'stwd_eng', 'stwd_kor', '영문약어명', '용어명 분리', 'eng_cnt']
df_step_1 = df_step_1[df_step_1_list]

# 비교군 저장
df_step_1_chk = df_step_1.copy()
df_step_1.head()

# 2차
# 널 제외 한글단어를 표준화된 풀데이터(df_step_1)를 다시검색하여 영문명 자동 기입 : 없는 것은 수동기입 필요 

# 영문 null 값 '' 채우기
df_step_1['stwd_eng'] = df_step_1['stwd_eng'].fillna("")

df_step_1['eng_cor_yn'] = ""
for i in range(df_step_1.shape[0]):
    blnk_chk = df_step_1.loc[i, 'stwd_eng'] == ''
    if blnk_chk :
        df_step_1.loc[i, 'stwd_eng'] = '!' + '_'.join(set(stwd_total.loc[(stwd_total['stwd_kor'] == df_step_1.loc[i, 'stwd_kor']), 'stwd_eng'].to_list()))
        df_step_1.loc[i, 'eng_cor_yn'] = 'n'

df_step_1_chk = df_step_1.copy()
        
# 순서
df_step_1_xlsx = df_step_1.sort_values(['stwd_eng'])
df_step_1_xlsx = df_step_1_xlsx[df_step_1_xlsx['eng_cor_yn'] == 'n']
df_step_1_xlsx.to_excel(output_path+"chk_eng" + ".xlsx")

# # 파일 실행 하지 않음
# os.system(output_path + "chk_eng.xlsx")

print(df_step_1_xlsx.shape)
df_step_1_xlsx.head()

### 6.2. 수정 후 재업로드(한글): 개발 미정

### 6.3. 수정 후 재업로드(영문)

#### 6.3. 영문수정 > re_load 만 탐색할 수 있도록 코드 간략화 필요(지금은 시간상 그냥 개발)

#### 6.3.1. 자동수정 후 재업로드(영문)

In [ ]:
# 최초 1회 수정 후 재업로드

# 수기 수정된 데이터
######## 가짜, 추후(chk_eng)로 수정 ########
re_load = pd.read_excel(output_path + 'chk_eng.xlsx', index_col='Unnamed: 0')

# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor = df_step_1.copy()

# 모두null일 경우(=전부기입된경우), 그냥 진행, 그렇지 않을 경우 수정된 값 적용 후 진행ㅇ
# Index 병합, 수기수정 대입, 그 외 null 지정
if re_load['stwd_eng'].isna().all():
    print("검수 통과")
else: 
    df_step_1_cor['stwd_eng'] =  re_load['stwd_eng']
    print("수기 수정 다시")
    
# 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
for i in range(df_step_1_cor.shape[0]):
    if pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        df_step_1_cor.loc[i, 'stwd_eng'] = df_step_1.loc[i, 'stwd_eng']
        df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'

    if df_step_1_cor.loc[i, 'eng_cor_yn'] == 'n':
        if df_step_1_cor.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'
        elif df_step_1_cor.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'n'

# 특수문자, 빈칸, null 점검
tmp_list = []
for i in range(df_step_1_cor.shape[0]):
    if len(re.findall('[^A-Z0-9+]', df_step_1_cor.loc[i, 'stwd_eng'])):
        tmp_list.append(i)
    elif df_step_1_cor.loc[i, 'stwd_eng'] == '':
        tmp_list.append(i)
    elif pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        tmp_list.append(i)

# 특수문자 'n'으로
df_step_1_cor.loc[tmp_list, 'eng_cor_yn'] = 'n'

# 전체 파일 저장
df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")

# 영문 점검 파일 저장    
chk_eng = df_step_1_cor.loc[df_step_1_cor['eng_cor_yn'] == 'n']
chk_eng.to_excel(output_path + "chk_eng.xlsx")

# 파일 실행
os.system(output_path + "chk_eng.xlsx")

print(df_step_1_cor.shape)
df_step_1_cor.head()

#### 6.3.2. 수기수정 후 재업로드(영문)

In [ ]:
# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor_tmp = df_step_1_cor.copy()
# 2회 이상부터

def repeat_eng():
    global df_step_1_cor_tmp
    global chk_eng
    global tmp_list_eng
#     global re_load

    # 수기 수정 후 다시 업로드: na_filter=False
    re_load = pd.read_excel(output_path + 'chk_eng.xlsx', index_col='Unnamed: 0', na_filter=False)
    
    # Index 병합, 수기수정 대입, 그 외 ''null'' 지정
    for i in re_load.index:
        df_step_1_cor_tmp.loc[i, 'stwd_eng'] =  re_load.loc[i, 'stwd_eng']
    
    # 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
    for i in range(df_step_1_cor_tmp.shape[0]):
        if df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] == 'n':
            if df_step_1_cor_tmp.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'
            elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'n'
                
        if pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            df_step_1_cor_tmp.loc[i, 'stwd_eng'] = df_step_1_cor.loc[i, 'stwd_eng']
            df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'

    # 특수문자, 빈칸, 소문자, 한글, null 점검
    tmp_list_eng = []
    for i in range(df_step_1_cor_tmp.shape[0]):
        if len(re.findall('[^A-Z0-9+]', df_step_1_cor_tmp.loc[i, 'stwd_eng'])):
            tmp_list_eng.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == '':
            tmp_list_eng.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            tmp_list_eng.append(i)

    # 특수문자 'n'으로
    df_step_1_cor_tmp.loc[tmp_list_eng, 'eng_cor_yn'] = 'n'

    # 파일 실행
    os.system(output_path + "chk_eng.xlsx")

    # 파일 저장
    chk_eng = df_step_1_cor_tmp.loc[df_step_1_cor_tmp['eng_cor_yn'] == 'n']
    df_step_1_cor_tmp.to_excel(output_path + "df_step_1_cor.xlsx")
    chk_eng.to_excel(output_path + "chk_eng.xlsx")

#### 6.3.3. 재업로드 반복 실행

In [ ]:
# 한글명, 영문명이 모두 ''값일 경우, 수정 완료
if (chk_eng[['stwd_kor', 'stwd_eng']] == '').all().all():
    print('"영문 수정 완료"' + " (빈칸 수:" + str(sum(df_step_1_cor_tmp['stwd_eng'] == '')) + ")")
    repeat_eng()
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("영문명 수정이 필요합니다. 반복 점검하시겠습니까?"):
        repeat_eng()
    else:
        os.system(output_path + "chk_eng.xlsx")    

In [ ]:
# 함수화 (추후에)

# def repeat_eng():
#     global df_step_1_cor_tmp
#     global chk_eng
#     global tmp_list_eng

In [ ]:
# #다시 검사(한글 1회차)

# df_step_1_cor = df_step_1_cor_tmp.copy()

# df_step_1_cor['kor_cor_yn'] = 'n'

# # 한글 검사
# tmp_list=[]
# for i in range(df_step_1_cor.shape[0]):
#     # 영어가 들어갔따
#     if (len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1):
#         tmp_list.append(i)
#         print(i)
# che_kor = df_step_1_cor.loc[tmp_list]
# che_kor.to_excel(output_path + "chk_kor.xlsx")

# # 수기 수정 후 다시 업로드: na_filter=False
# re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)

# os.system(output_path + "chk_kor.xlsx")

# re_load

In [ ]:
#다시 검사(한글 1회차)

df_step_1_cor = df_step_1_cor_tmp.copy()

# kor_cor_yn 초기값 'n' 설정
df_step_1_cor['kor_cor_yn'] = ''

# 한글 검사
tmp_list=[]

for i in range(df_step_1_cor.shape[0]):
    cdtn_eng = len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1 # 영문이 한자라도 있을 시
    cdtn_kor = len(re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1 # 한글이 한자라도 있을 시
    cdtn_eng_lower = len(re.findall('[a-z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
    # 한글과 영문이 섞인 경우만: 대문자만 들어갈 경우 허용
    if cdtn_eng & cdtn_kor:
        tmp_list.append(i)
        print(i)
    # 소문자가 들어간 경우만
    elif cdtn_eng_lower:
        tmp_list.append(i)
        print(i)

df_step_1_cor.loc[tmp_list, 'kor_cor_yn'] = 'n'
chk_kor = df_step_1_cor.loc[tmp_list]
chk_kor.to_excel(output_path + "chk_kor.xlsx")

# 수기 수정 후 다시 업로드: na_filter=False
re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)

os.system(output_path + "chk_kor.xlsx")

re_load

In [ ]:
#     # 한글 검사
#     tmp_list=[]
#     for i in range(df_step_1_cor.shape[0]):
#         cdtn_eng = len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         cdtn_kor = len(re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         cdtn_eng_lower = len(re.findall('[a-z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         # 영문 대문자만 들어간 경우 허용
#         if cdtn_eng:
#             tmp_list.append(i)
#             print(i)
#         # 한글 단어만 들어간 경우 허용
#         elif cdtn_kor:
#             tmp_list.append(i)
#             print(i)
#     # 올바르게 적용된 내용 'y'로
#     df_step_1_cor_tmp.loc[tmp_list_kor, 'kor_cor_yn'] = 'y'

# 다시다시... y n 바꾸는거 안됨..

In [ ]:
#다시 검사(한글 n회차)

def repeat_kor():
    global df_step_1_cor
    global chk_kor
    global tmp_list_kor

    # 수기 수정 후 다시 업로드: na_filter=False
    re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)
    re_load

    # 엑셀 수정내용 적용
    for i in range(df_step_1_cor.shape[0]):
        for j in re_load.index:
            if df_step_1_cor.index.to_list()[i] == j:
                df_step_1_cor.loc[i, 'stwd_kor'] = re_load.loc[j, 'stwd_kor']

    # 비교하기위한 복사
    df_step_1_cor_tmp = df_step_1_cor.copy()

    # 그외 null 값은 이전 데이터(df_step_1_cor) 값으로 채워넣기
    for i in range(df_step_1_cor_tmp.shape[0]):
        if df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] == 'n':
            # 이전 데이터와 비교하여 바뀐값 'y' 입력
            if df_step_1_cor_tmp.loc[i, 'stwd_kor'] != df_step_1_cor.loc[i, 'stwd_kor']:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
            # 이전 데이터와 비교하여 바뀌지 않은 값 'n' 입력
            elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == df_step_1_cor.loc[i, 'stwd_kor']:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'n'
        
        # null일경우 '' 입력
        if pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = ''

    # 특수문자, 빈칸, 소문자, 한글, null 점검
    tmp_list_kor = []
    for i in range(df_step_1_cor_tmp.shape[0]):
        
        # 수정필요(n)사항 중에서..
        if df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] == 'n':
            cdtn_eng_len = re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])
            cdtn_kor_len = re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])
            # 한글만 한자 이상
            if cdtn_eng == 0 & cdtn_kor >= 1:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
                print(i)
            # 영문만 한자 이상
            elif cdtn_eng >= 1 & cdtn_kor == 0:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
                print(i)
            
        # 소문자가 들어간 경우만
        elif cdtn_eng_lower:
            tmp_list_kor.append(i)
            print(i)
            
        # 허용값이 아닌 개수 1개 이상
        elif len(re.findall('[^가-힣A-z0-9+]', df_step_1_cor_tmp.loc[i, 'stwd_kor'])): 
            tmp_list_kor.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == '':
            tmp_list_kor.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            tmp_list_kor.append(i)

    # 특수문자 'n'으로
    df_step_1_cor_tmp.loc[tmp_list_kor, 'kor_cor_yn'] = 'n'
        
    chk_kor = df_step_1_cor_tmp.loc[df_step_1_cor_tmp['kor_cor_yn'] == 'n']
    chk_kor.to_excel(output_path + "chk_kor.xlsx")
    
    df_step_1_cor = df_step_1_cor_tmp
    df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    
    os.system(output_path + "chk_kor.xlsx")

In [ ]:
repeat_kor()

In [ ]:
df_step_1_cor

In [ ]:
df_step_1_cor_tmp

In [ ]:
# 동작 안됨
# 한글명, 영문명이 모두 ''값일 경우, 수정 완료
if (chk_kor[['stwd_kor', 'stwd_eng']] == '').all().all():
    print('"한글 수정 완료"' + " (빈칸 수:" + str(sum(df_step_1_cor['stwd_kor'] == '')) + ")")
    repeat_kor()
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("한글명 수정이 필요합니다. 반복 점검하시겠습니까?"):
        repeat_kor()
    else:
        os.system(output_path + "chk_kor.xlsx")    

In [ ]:
df_step_1_cor_tmp

In [ ]:
# 영문 중복 검사
if sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1: # keep=False: 중복중 하나만 유지하지 않고 중복인 경우 모두 체크한다
    print("영문명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
# 한글 중복 검사
elif sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1:
    print("한글명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
else:
    print("단어 중복을 모두 확인하였습니다.")
    
#     #코드 작성(적용)
#     df_step_1_cor = df_step_1_cor_tmp
#     df_step_1_cor.to_excel(output_path +"df_step_1_cor.xlsx")
    
#     print("무의미한 값('')을 삭제하였습니다.")
#     df_space = df_step_1_cor_dup.loc[(df_step_1_cor_dup['stwd_eng'] == '') | (df_step_1_cor_dup['stwd_kor'] == '')]
#     n_space_list = list(set(df_step_1_cor_dup.index) - set(df_space.index))
#     df_step_1_cor = df_step_1_cor.loc[n_space_list]
#     df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    
#     print("단어 사전을 완성하였습니다.")
    os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
df_step_1_cor_kor_dup = df_step_1_cor[df_step_1_cor['stwd_kor'].duplicated(keep=False)]
df_step_1_cor_kor_dup.to_excel(output_path + "df_step_1_cor_kor_dup.xlsx")
df_step_1_cor_kor_dup

In [ ]:

# 영문 중복 검사
if sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1: # keep=False: 중복중 하나만 유지하지 않고 중복인 경우 모두 체크한다
    print("영문명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
# 한글 중복 검사
elif sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1:
    print("한글명 중복을 탐지하였습니다.")
    
    
    
    #코드 작성
    
else:
    
    
    print("단어 중복을 모두 확인하였습니다.")
    print("무의미한 값('')을 삭제하였습니다.")
    df_space = df_step_1_cor.loc[(df_step_1_cor['stwd_eng'] == '') | (df_step_1_cor['stwd_kor'] == '')]
    n_space_list = list(set(df_step_1_cor.index) - set(df_space.index))
    df_step_1_cor = df_step_1_cor.loc[n_space_list]
    df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    print("단어 사전을 완성하였습니다.")
    os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
df_step_1_cor_dup = df_step_1_cor[df_step_1_cor['stwd_kor'].duplicated(keep=False)]
df_step_1_cor_dup.to_excel(output_path + "df_step_1_cor_dup.xlsx")
df_step_1_cor_dup

In [ ]:
df_step_1_cor_dup = pd.read_excel(output_path + "df_step_1_cor_dup.xlsx")
# df_step_1_cor_dup.to_excel(output_path + "df_step_1_cor_dup.xlsx")
df_step_1_cor_dup

In [ ]:
df_step_1_cor_dup['stwd_kor'].duplicated(keep=False)

In [ ]:
# 엑셀 수정내용 적용
for i in range(df_step_1_cor.shape[0]):
    for j in df_step_1_cor_dup.index:
        if df_step_1_cor.index.to_list()[i] == j:
            df_step_1_cor.loc[i, 'stwd_kor'] = df_step_1_cor_dup.loc[j, 'stwd_kor']

In [ ]:
df_step_1_cor

In [ ]:
df_step_1_cor_dup.index

In [ ]:
sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1

In [ ]:
sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1

In [ ]:
df_step_1_cor.loc[df_step_1_cor['stwd_kor'].duplicated(keep=False)]

In [ ]:
[단어]
[v]한글명이상 검사 (예: 영문 대문자)
[v]한글 하나,영문 하나


[]다시, 한글 영어 중복검사
[]단어 순서 검사

[]스페이스 제거하여 완전 사전?

<용어>
[]단어 개수 검사 (가A ID 생성 후 일치화)



In [ ]:
os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
stwd_kor

In [ ]:
            
        # 한국어 점검
        if len(re.findall('[^A-Z0-9가-힣+]', df_step_1_cor_tmp.loc[i, 'stwd_kor'])):
            tmp_list_eng.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == '':
            tmp_list_eng.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            tmp_list_eng.append(i)

In [ ]:
def eng_blnk_chk():
    global chk_eng
    global df_step_1_cor_tmp
    
    # count 점검: 빈칸은 한개만 허용함
    cnt_chk = sum(df_step_1_cor_tmp.loc[tmp_list, 'stwd_eng'] == '')
    if cnt_chk == 1:
        print("빈칸 허용치와 일치: " + str(cnt_chk))
        if sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] !=  '') == 0:
            print('"영문 수정 완료"')
        else:
            print('추가 수정 필요')
    elif cnt_chk < 1:
        print("빈칸이 허용치보다 적음: " + str(cnt_chk))
    elif cnt_chk >= 2:
        print("빈칸이 허용치보다 많음: " + str(cnt_chk))

In [ ]:
여기에 의존하면 안돼..

In [ ]:
# '' == 1개, 'n' 중에서 ''가 하나도 없을 경우 작업 종료 
condition_1 = sum(df_step_1_cor_tmp.loc[tmp_list, 'stwd_eng'] == '') == 1
# condition_2 = sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] !=  '') == 0
condition_2 = sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] == '') == 1
if condition_1 and condition_2:
    print("더 이상 진행할 수 없습니다.")
    os.system(output_path + "chk_eng.xlsx")
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("반복 점검하시겠습니까?"):
        repeat_eng()
        eng_blnk_chk()
    else:
        os.system(output_path + "chk_eng.xlsx")

In [ ]:
df_step_1 = df_step_1_cor_tmp[df_step_1_cor_tmp['stwd_eng'].notna()]
df_step_1 = df_step_1_cor_tmp[df_step_1_cor_tmp['stwd_eng'] != '']

In [ ]:
# count 확인
find_unique_eng(df_step_1)

In [ ]:
# count 확인

if sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_')) == 0:
    print('영문명을 모두 수정하였습니다 !_수: '+ str(sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_'))))
    print(df_step_1_cor_tmp.shape)
else:
    print('아직 수정하지 않은 영문명이 남아있습니다 !_수: '+ str(sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_'))))    
    print(df_step_1_cor_tmp.shape)
    
# 두 카운트가 같아아 최종 표준화 점검 완료 1:1 대응
if len(set(df_step_1['stwd_kor'])) == len(set(df_step_1['stwd_eng'])):
    print('영문명과 한글명의 고유개수가 일치합니다')
print("영문명: " + str(len(set(df_step_1['stwd_eng']))))
print("한글명: " + str(len(set(df_step_1['stwd_kor']))))

In [ ]:
print('_'.join(stwd_total.loc[stwd_total['index'] == 1055, 'stwd_kor'].to_list()))
stwd_total.loc[stwd_total['index'] == 1055, 'stwd_kor']

In [ ]:
df_step_1['stwd_kor'].isna()[i]

In [ ]:
df_step_1.to_excel(output_path+"chk_eng" + ".xlsx")

In [ ]:
len(set(df_step_1['stwd_eng']))

In [ ]:
df_step_1

In [ ]:
index = [1055, 1056, 1057, 445, 432, 724, 878, 959, 1011, 934, 781, 454, 239]

In [ ]:
stwd_total.loc[(stwd_total['stwd_eng'] == 'NUM') & (stwd_total['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
# 널 제외 한글단어를 표준화된 풀데이터(df_step_1)를 다시검색하여 영문명 자동 기입 : 없는 것은 수동기입 필요
for i in range(tmp_df_step_1.shape[0]):
    isna_chk = tmp_df_step_1['stwd_kor'].isna()[i]
    if isna_chk:
#         tmp_df_step_1.loc[i, 'stwd_kor'] = stwd_total.loc[(stwd_total['stwd_eng'] == 'NUM') & (stwd_total['stwd_kor'] != ''), 'stwd_kor'].values[0]
        tmp_df_step_1.loc[i, 'stwd_kor'] = df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

    
# df_step_1_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
# tmp_df_step_1 = tmp_df_step_1[df_step_1_list]
# print(tmp_df_step_1.shape)
# tmp_df_step_1.head()
std_chk_eng

# 순서
tmp_df_step_1 = tmp_df_step_1.sort_values(['stwd_eng'])
tmp_df_step_1.to_excel(output_path+"chk_eng" + ".xlsx")

print(tmp_df_step_1.shape)
tmp_df_step_1

In [ ]:
null_cnt = 0
if null_cnt > 0:
    print("빈칸이 있습니다")
### ㅅ수정 후 재업로드 하여 null count
# ''개수가 1 이상일 경우 일경우 되묻지 않고 안내만
# ''개수가 0일경우 되묻지 않고 안내만


In [ ]:
stwd_total.loc[stwd_total['index'] == 781, 'stwd_kor'].values

In [ ]:
index = [1055, 1056, 1057, 445, 432, 724, 878, 959, 1011, 934, 781, 454, 239]
for i in range(len(index)):
    a = stwd_total.loc[stwd_total['index'] == index[i], 'stwd_kor']
    print(a)
    
a

In [ ]:
df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
stwd_total.loc[stwd_total['stwd_eng'] == 'HAKJUM', 'stwd_kor'].values[0]

In [ ]:
# 한국어가 null값이면 자동 영문명 자동 기입


In [ ]:
tmp_df_step_1

In [ ]:
tmp_df_step_1

In [ ]:







print(df_step_1.shape)
df_step_1.head()

In [ ]:
std_chk

In [ ]:
std_chk

In [ ]:



a.to_excel(output_path + "dfsfd.xlsx")
# dup_cnt_eng

#     dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
#     globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)

        
# for i in range(eng_firts_cnt):
#     globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
#     globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]

In [ ]:
df_step_1['stwd_eng'].to_list().count(df_step_1.loc[0, 'stwd_eng'])

In [ ]:
dupl_list_eng0['dupl_list_eng0'].to_list().count(1)

In [ ]:
tmp_df_step_1.shape[0]

In [ ]:
# 중복 지우기 위해


In [ ]:
# 널 채우기 위해
tmp_df_step_1[stwd_kor]

In [ ]:
df_step_1.shape

In [ ]:
df_step_1.to_excel(input_path+"111.xlsx")
std_chk.to_excel(input_path+"112.xlsx")

In [ ]:
# public_stwd_cnt =public_stwd.shape[0]
# df_step_1_cnt = df_step_1.shape[0]

# for i in range(df_step_1_cnt):
#     for j in range(public_stwd_cnt):
#         if df_step_1.iloc[i,1] == public_stwd.iloc[j,1]: #[i, 0]
#             # 영문명 수정
#             df_step_1.iloc[i,3] = public_stwd.iloc[j,2]
#         else:
#             a = 1
# #             df_step_1.iloc[i,1] = "그대로"+df_step_1.iloc[i,1]

In [ ]:
# 파일 저장 묻기
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')    

if yes_or_no('파일을 저장하시겠습니까?'):
    file_name = 'stwd_total_manu1' + "_" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S') + ".xlsx")
    print('[엑셀 파일을 생성하였습니다]: ' + file_name )
else:
    print('[검토한 후 다시 업로드 해주세요]')
stwd_total.head()